# Preprocessing:
1. Building the benchmark dataset by combining all five datasets.
2. Padding images to generate squared images.
3. preparing the Multitask dataset


### Required packages:
1. install openpyxl by running: !pip install openpyxl
2. Pandas, numpy, matplotlib, Pillow


In [14]:
import os
import pandas as pd
import numpy as np 
from os import listdir
from PIL import Image
import matplotlib.pyplot as plt

## 1. Prepare the HMSS dataset


In [15]:
# Reading data

df_HMSS = pd.read_csv('Datasets/HMSS/HMSS.csv')
Path_hmss = 'Datasets/HMSS/imgs/'
imgs_path_hmss = Path_hmss + df_HMSS['img name']
imgs_name_hmss = list(df_HMSS['img name'])
imgs_path_hmss = list(imgs_path_hmss)
imgs_cover_hmss = df_HMSS['case id/cover img id']
tumor_type_hmss = df_HMSS['tumor type']


## 2. Prepare the BUSIS dataset



In [16]:
# Reading data
df_BUSIS = pd.read_csv('./Datasets/BUSIS/BUSIS562.csv')
Path_busis = 'Datasets/BUSIS/imgs/'
imgs_path_busis = Path_busis+df_BUSIS["img name"]
imgs_name_busis = list(df_BUSIS["img name"])
imgs_path_busis = list(imgs_path_busis)
imgs_cover_busis = list(imgs_name_busis)
tumor_type_busis = list(df_BUSIS["tumor type"])
# print(cover_image_busis)

## 3. Prepare the DatasetB

In [17]:
# Reading data
df_DatasetB = pd.read_excel('./Datasets/DatasetB/DatasetB.xlsx', converters={'Image': lambda x: str(x)})
Path_busis = 'Datasets/DatasetB/imgs/'
imgs_path_DatasetB = Path_busis+df_DatasetB["Image"].apply(str)
imgs_name_DatasetB = list(df_DatasetB["Image"].apply(str))
tumor_type_DatasetB = list(df_DatasetB["Type"].apply(str))
# print(str(df_DatasetB))
imgs_cover_DatasetB = imgs_name_DatasetB
imgs_path_DatasetB = list(imgs_path_DatasetB)


### In case of more than one tumor in a BUS image, the BUSI dataset created a mask for each tumor. To solve this issue, we combine them to a single mask file. 

In [18]:

from numpy import asarray

def merge_masks(combined_img_savepath_mask):
    
    Cases_multiple_tumors = { 'benign (100)_mask.png':['benign (100)_mask_1.png']                                              
     ,'benign (163)_mask.png':['benign (163)_mask_1.png']                                              
     ,'benign (173)_mask.png':['benign (173)_mask_1.png']                                              
     ,'benign (181)_mask.png':['benign (181)_mask_1.png']                                              
     ,'benign (195)_mask.png':['benign (195)_mask_1.png', 'benign (195)_mask_2.png']                                               
     ,'benign (25)_mask.png': ['benign (25)_mask_1.png']                                               
    , 'benign (315)_mask.png': ['benign (315)_mask_1.png']                                              
     ,'benign (346)_mask.png': ['benign (346)_mask_1.png']                                              
     ,'benign (4)_mask.png':['benign (4)_mask_1.png']                                                
     ,'benign (424)_mask.png':['benign (424)_mask_1.png']                                              
     ,'benign (54)_mask.png':['benign (54)_mask_1.png']                                               
     ,'benign (58)_mask.png':['benign (58)_mask_1.png']                                               
     ,'benign (83)_mask.png':['benign (83)_mask_1.png']                                               
     ,'benign (92)_mask.png':['benign (92)_mask_1.png']                                               
     ,'benign (93)_mask.png':['benign (93)_mask_1.png']                                               
     ,'benign (98)_mask.png':['benign (98)_mask_1.png']                                               
     ,'malignant (53)_mask.png':['malignant (53)_mask_1.png'] }

    imagesList_dup = listdir(combined_img_savepath_mask)
    for x in imagesList_dup:
        for key, value in Cases_multiple_tumors.items():
            if key == x:
                key_obj = Image.open(os.path.join(combined_img_savepath_mask, key ))
                key_obj = key_obj.convert("L")
                key_arr = asarray(key_obj)
                print("Key object = ", key_obj, key, key_arr.shape)
                for i in range(len(value)):
                    value_obj = Image.open(os.path.join(combined_img_savepath_mask, value[i] ))

                    #print(value_obj.shape)
                    value_obj = value_obj.convert("L")
                    value_arr = asarray(value_obj)
                    print(value_arr.shape, key_arr.shape, value)

                    key_arr =  np.add(key_arr, value_arr)
                        # remove image
                    os.remove(os.path.join(combined_img_savepath_mask, value[i] ))
                    print(value_arr.shape, key_arr.shape)
                img_obj = Image.fromarray(key_arr)
                img_obj.save(os.path.join(combined_img_savepath_mask, key ))

### 4. Prepare the BUSI dataset
BUSI dataset consist of "malignant", "benign", and "normal" folders. We use only "malignant" and "benign" folders and combine them. Then we create a "BUSI.csv" file which contain image names and their classification ground truth. We also create a folder for segmentation masks.  

In [19]:
Path_busi_ben = 'Datasets/BUSI/benign/'
Path_busi_mal = 'Datasets/BUSI/malignant/'
save_csv_busi = 'Datasets/BUSI/'

combined_img_savepath = 'Datasets/BUSI/BUSI_combined/imgs/'                # A directory to save BUS images
combined_masks_savepath = 'Datasets/BUSI/BUSI_combined/masks/'            # A directory to save segmentation masks
if not os.path.exists(combined_img_savepath):
    os.makedirs(combined_img_savepath)
    
if not os.path.exists(combined_masks_savepath):
    os.makedirs(combined_masks_savepath)

imagesList1 = listdir(Path_busi_ben)
imagesList2 = listdir(Path_busi_mal)

substring ="mask"
print(len(imagesList1), len(imagesList2))
# df_ori.head()
benign_list=[]
benign_class =[]
malignant_list=[]
malignant_class = []

for imageName in imagesList1:
    if substring not in imageName:
        img_obj = Image.open(os.path.join(Path_busi_ben, imageName ))
        img_obj.save(os.path.join(combined_img_savepath, imageName ))

        benign_list.append(imageName[:-4])
        benign_class.append(0)
    else:
        img_obj = Image.open(os.path.join(Path_busi_ben, imageName ))
        img_obj.save(os.path.join(combined_masks_savepath, imageName ))

        
for imageName in imagesList2:
    if substring not in imageName:
        img_obj = Image.open(os.path.join(Path_busi_mal, imageName ))
        img_obj.save(os.path.join(combined_img_savepath, imageName ))
        malignant_list.append(imageName[:-4])
        malignant_class.append(1)
    else:
        img_obj = Image.open(os.path.join(Path_busi_mal, imageName ))
        img_obj.save(os.path.join(combined_masks_savepath, imageName ))

        
merge_masks(combined_masks_savepath)

#    malignant_list.append(imageName)
df = pd.DataFrame(list(zip(benign_list, benign_class)),
                  columns=['img name', 'tumor type'])
df2 = pd.DataFrame(list(zip(malignant_list, malignant_class)),
                  columns=['img name', 'tumor type'])
df3 = df.append(df2,ignore_index=True) 
df3.to_csv(os.path.join(save_csv_busi, 'BUSI.csv'), index=False)
imgs_path_busi = combined_img_savepath+df3["img name"]

masks_path_busi = combined_img_savepath+df3["img name"]

imgs_name_busi = list(df3["img name"].apply(str))
imgs_path_busi = list(imgs_path_busi)
imgs_cover_busi = imgs_name_busi
tumor_type_busi = list(df3["tumor type"].apply(str))


891 421
Key object =  <PIL.Image.Image image mode=L size=323x473 at 0x156B11338B0> benign (100)_mask.png (473, 323)
(473, 323) (473, 323) ['benign (100)_mask_1.png']
(473, 323) (473, 323)
Key object =  <PIL.Image.Image image mode=L size=779x576 at 0x156B1133400> benign (163)_mask.png (576, 779)
(576, 779) (576, 779) ['benign (163)_mask_1.png']
(576, 779) (576, 779)
Key object =  <PIL.Image.Image image mode=L size=726x573 at 0x156B1133A60> benign (173)_mask.png (573, 726)
(573, 726) (573, 726) ['benign (173)_mask_1.png']
(573, 726) (573, 726)
Key object =  <PIL.Image.Image image mode=L size=733x455 at 0x156B1133A30> benign (181)_mask.png (455, 733)
(455, 733) (455, 733) ['benign (181)_mask_1.png']
(455, 733) (455, 733)
Key object =  <PIL.Image.Image image mode=L size=729x611 at 0x156B1133400> benign (195)_mask.png (611, 729)
(611, 729) (611, 729) ['benign (195)_mask_1.png', 'benign (195)_mask_2.png']
(611, 729) (611, 729)
(611, 729) (611, 729) ['benign (195)_mask_1.png', 'benign (195)_m

C:\Users\shar0416\AppData\Local\Temp\ipykernel_20780\3594654195.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df.append(df2,ignore_index=True)


### 4. Prepare the Thammasat dataset

In [20]:
df_Thamm = pd.read_csv('./Datasets/Thammasat/Thammasat.csv')
Path_Thamm = 'Datasets/Thammasat/imgs/'

imgs_path_Thamm = Path_Thamm + df_Thamm["img name"]
imgs_name_Thamm = list(df_Thamm["img name"].apply(str))
imgs_path_Thamm = list(imgs_path_Thamm)
imgs_cover_Thamm = list(df_Thamm["case id/cover img id"].apply(str))
tumor_type_Thamm = list(df_Thamm["tumor type"].apply(str))
# print(tumor_type_Thamm)


### 5. Combine all datasets
We combine the BUS images of the five datasets and put it under 'Datasets\BUS_Combined\imgs'. Next, the file names and their tumor categories are saved in 'BUS_Combined.csv' file under 'Datasets\BUS_Combined\'. 

In [21]:
save_csv_benchmark = 'Datasets\BUS_Combined'
if not os.path.exists(save_csv_benchmark):
    os.makedirs(save_csv_benchmark)

names_list=[imgs_name_hmss, imgs_name_DatasetB, imgs_name_busis,imgs_name_busi,imgs_name_Thamm]
paths_list=[imgs_path_hmss, imgs_path_DatasetB, imgs_path_busis,imgs_path_busi,imgs_path_Thamm]
cover_list=[imgs_cover_hmss, imgs_cover_DatasetB, imgs_cover_busis,imgs_cover_busi,imgs_cover_Thamm]
tumor_type_list=[tumor_type_hmss, tumor_type_DatasetB, tumor_type_busis,tumor_type_busi,tumor_type_Thamm]
ext_list=['.jpg', '.png', '.png','.png','.jpg']

### Multitask 
names_imgs_multi=[imgs_name_busis,imgs_name_busi]
paths_imgs_multi=[imgs_path_busis,imgs_path_busi]
ext_list2=[ '.bmp', '.png']

####
df_hmss = pd.DataFrame(list(zip(imgs_name_hmss, imgs_cover_hmss,tumor_type_hmss)),
                       columns=['img name', 'case id/cover img id', 'tumor type'])
df_DB = pd.DataFrame(list(zip(imgs_name_DatasetB, imgs_cover_DatasetB,tumor_type_DatasetB)),
                     columns=['img name', 'case id/cover img id', 'tumor type'])
df_busis = pd.DataFrame(list(zip(imgs_name_busis, imgs_cover_busis,tumor_type_busis)),
                        columns=['img name', 'case id/cover img id', 'tumor type'])
df_busi = pd.DataFrame(list(zip(imgs_name_busi, imgs_cover_busi,tumor_type_busi)),
                       columns=['img name', 'case id/cover img id', 'tumor type'])
df_Thamm = pd.DataFrame(list(zip(imgs_name_Thamm, imgs_cover_Thamm,tumor_type_Thamm)),
                        columns=['img name', 'case id/cover img id', 'tumor type'])
dfs3 = df_hmss.append(df_DB,ignore_index=True)          
dfs3 = dfs3.append(df_busis,ignore_index=True)          
dfs3 = dfs3.append(df_busi,ignore_index=True) 
dfs3 = dfs3.append(df_Thamm,ignore_index=True)     
dfs3['tumor type'] = dfs3["tumor type"].apply(lambda x: 0 if x == 0 or x == "0" or x ==  "B" or x =="b" or x == "benign" or x =="Benign" else 1)       
dfs3['case id/cover img id'] = dfs3['case id/cover img id']
dfs3['img name'] = dfs3['img name']
dfs3.to_csv(os.path.join(save_csv_benchmark, 'BUS_Combined.csv'), index=False)
print(dfs3[2290:2300])


      img name case id/cover img id  tumor type
2290  case0122             case0122           0
2291  case0123             case0123           0
2292  case0124             case0124           0
2293  case0125             case0125           0
2294  case0126             case0126           0
2295  case0127             case0127           0
2296  case0128             case0128           0
2297  case0129             case0129           0
2298  case0130             case0130           0
2299  case0131             case0131           0


C:\Users\shar0416\AppData\Local\Temp\ipykernel_20780\2147738680.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfs3 = df_hmss.append(df_DB,ignore_index=True)
C:\Users\shar0416\AppData\Local\Temp\ipykernel_20780\2147738680.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfs3 = dfs3.append(df_busis,ignore_index=True)
C:\Users\shar0416\AppData\Local\Temp\ipykernel_20780\2147738680.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfs3 = dfs3.append(df_busi,ignore_index=True)
C:\Users\shar0416\AppData\Local\Temp\ipykernel_20780\2147738680.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfs3 = dfs3.append(df_Thamm,ignore_i

### 6. Image padding

 Padding methods:
 We apply zero-padding either horizontally or vertically to all the images. 

In [22]:
def Pad_left_right(img_obj, left_to_pad,right_to_pad, height):
    
    img_obj_padded = Image.new('RGB', (height, height))
    img_obj_padded.paste(left_to_pad, (0, 0))
    img_obj_padded.paste(img_obj, (left_to_pad.width, 0))
    img_obj_padded.paste(right_to_pad, (left_to_pad.width+img_obj.width, 0))
    
    return img_obj_padded

def Pad_top_bottom(img_obj, top_to_pad, bottom_to_pad, width):
    
    img_obj_padded = Image.new('RGB', (width, width))
    img_obj_padded.paste(top_to_pad, (0, 0))
    img_obj_padded.paste(img_obj, (0,top_to_pad.height))
    img_obj_padded.paste(bottom_to_pad, (0, top_to_pad.height+(img_obj.height)))
    
    return img_obj_padded

def Pad_right(img_obj, right_to_pad, height):
    
    img_obj_padded = Image.new('RGB', (height, height))
    #img_obj_padded.paste(img_obj, (left_to_pad.width, 0))
    img_obj_padded.paste(right_to_pad, (img_obj.width, 0))
    
    return img_obj_padded

def Pad_bottom(img_obj, obj_to_pad, width):
    img_obj_padded = Image.new('RGB', (width, width))
    img_obj_padded.paste(img_obj, (0, 0))
    img_obj_padded.paste(obj_to_pad, (0, img_obj.height))
   
    return img_obj_padded

In [86]:
def preprocess(imgs_name, imgs_path, ext):
    for b in range(len(imgs_name)):
        print(len(imgs_name))
        filename = imgs_name[b]
        print("-----file name:", filename, b)
        print(imgs_path[b])
        img_obj = Image.open(os.path.join(imgs_path[b]+ext))
        img_np = np.array(img_obj)      
        #Remove white boundary of the image
        #img_np1 = img_np[1:-1, 1:-1]
        img_np2 = Image.fromarray(img_np)

        width, height = img_np2.size
        print(">>> Original width & height:", width, height, img_obj.size)

        dif = abs(width-height)
        if width-1 > height:
            topadd = int(dif/2)
            bottomadd = dif - topadd
            print("topadd = ", topadd, bottomadd)

            top_to_pad = Image.fromarray(np.zeros((topadd, width, 3), dtype= 'uint8'))
            bottom_to_pad = Image.fromarray(np.zeros((bottomadd, width,  3), dtype = 'uint8'))
            print("top_to_pad",top_to_pad.size, bottom_to_pad.size)

            img_obj_padded = Pad_top_bottom(img_np2, top_to_pad, bottom_to_pad, width)

        elif width < height-1:
            print(">>> Vertial: left/right pad with {} cols".format(dif))
            if dif == abs(1):
                leftadd = int(dif)
                rightadd = dif - leftadd
                
                left_to_pad = Image.fromarray(np.zeros((height, leftadd, 3), dtype = 'uint8'))
                #right_to_pad = Image.fromarray(np.zeros((height, rightadd, 3), dtype = 'uint8'))

                img_obj_padded = Pad_left_right(img_np2, left_to_pad, 0, height)

            else:
                leftadd = int(dif/2)
                rightadd = dif - leftadd
                
                left_to_pad = Image.fromarray(np.zeros((height, leftadd, 3), dtype = 'uint8'))
                right_to_pad = Image.fromarray(np.zeros((height, rightadd, 3), dtype = 'uint8'))

                img_obj_padded = Pad_left_right(img_np2, left_to_pad, right_to_pad, height)

        
        elif width == height-1: # just pad the single col to the right
            print(">>> Vertial: right pad with {} cols".format(dif))
            right_to_pad = Image.fromarray(np.zeros((height, 1, 3), dtype = 'uint8'))
            img_obj_padded = Pad_right(img_np2, right_to_pad, height)
            print(">>> After padding: ", img_obj_padded.size)

        else:
            print(">>> Originally square shape")
            img_obj_padded = img_np2

        filenames = os.path.splitext(filename)[0]
        img_obj_padded.save(os.path.join(img_savepath,filenames + '.png'))   

In [26]:
# Path to save the padded images
img_savepath = 'Datasets/BUS_Combined/imgs/'
if not os.path.exists(img_savepath):
     os.makedirs(img_savepath)
for i in range(len(names_list)):
    preprocess(names_list[i], paths_list[i],ext_list[i])



-----file name: 2879 0
Datasets/HMSS/imgs/2879
>>> Original width & height: 592 443 (592, 443)
topadd =  74 75
top_to_pad (592, 74) (592, 75)
-----file name: 3155 1
Datasets/HMSS/imgs/3155
>>> Original width & height: 432 364 (432, 364)
topadd =  34 34
top_to_pad (432, 34) (432, 34)
-----file name: 3156 2
Datasets/HMSS/imgs/3156
>>> Original width & height: 420 351 (420, 351)
topadd =  34 35
top_to_pad (420, 34) (420, 35)
-----file name: 3694 3
Datasets/HMSS/imgs/3694
>>> Original width & height: 596 415 (596, 415)
topadd =  90 91
top_to_pad (596, 90) (596, 91)
-----file name: 3695 4
Datasets/HMSS/imgs/3695
>>> Original width & height: 575 413 (575, 413)
topadd =  81 81
top_to_pad (575, 81) (575, 81)
-----file name: 3696 5
Datasets/HMSS/imgs/3696
>>> Original width & height: 594 361 (594, 361)
topadd =  116 117
top_to_pad (594, 116) (594, 117)
-----file name: 3697 6
Datasets/HMSS/imgs/3697
>>> Original width & height: 575 362 (575, 362)
topadd =  106 107
top_to_pad (575, 106) (575, 107

>>> Original width & height: 520 374 (520, 374)
topadd =  73 73
top_to_pad (520, 73) (520, 73)
-----file name: 5359 60
Datasets/HMSS/imgs/5359
>>> Original width & height: 421 378 (421, 378)
topadd =  21 22
top_to_pad (421, 21) (421, 22)
-----file name: 5360 61
Datasets/HMSS/imgs/5360
>>> Original width & height: 541 403 (541, 403)
topadd =  69 69
top_to_pad (541, 69) (541, 69)
-----file name: 5361 62
Datasets/HMSS/imgs/5361
>>> Original width & height: 485 328 (485, 328)
topadd =  78 79
top_to_pad (485, 78) (485, 79)
-----file name: 5362 63
Datasets/HMSS/imgs/5362
>>> Original width & height: 487 362 (487, 362)
topadd =  62 63
top_to_pad (487, 62) (487, 63)
-----file name: 5367 64
Datasets/HMSS/imgs/5367
>>> Original width & height: 477 403 (477, 403)
topadd =  37 37
top_to_pad (477, 37) (477, 37)
-----file name: 5368 65
Datasets/HMSS/imgs/5368
>>> Original width & height: 460 337 (460, 337)
topadd =  61 62
top_to_pad (460, 61) (460, 62)
-----file name: 5369 66
Datasets/HMSS/imgs/5369

top_to_pad (538, 71) (538, 72)
-----file name: 5444 117
Datasets/HMSS/imgs/5444
>>> Original width & height: 493 396 (493, 396)
topadd =  48 49
top_to_pad (493, 48) (493, 49)
-----file name: 5445 118
Datasets/HMSS/imgs/5445
>>> Original width & height: 558 355 (558, 355)
topadd =  101 102
top_to_pad (558, 101) (558, 102)
-----file name: 5446 119
Datasets/HMSS/imgs/5446
>>> Original width & height: 562 387 (562, 387)
topadd =  87 88
top_to_pad (562, 87) (562, 88)
-----file name: 5447 120
Datasets/HMSS/imgs/5447
>>> Original width & height: 467 334 (467, 334)
topadd =  66 67
top_to_pad (467, 66) (467, 67)
-----file name: 5448 121
Datasets/HMSS/imgs/5448
>>> Original width & height: 395 337 (395, 337)
topadd =  29 29
top_to_pad (395, 29) (395, 29)
-----file name: 5449 122
Datasets/HMSS/imgs/5449
>>> Original width & height: 480 330 (480, 330)
topadd =  75 75
top_to_pad (480, 75) (480, 75)
-----file name: 5451 123
Datasets/HMSS/imgs/5451
>>> Original width & height: 576 306 (576, 306)
topa

>>> Original width & height: 530 338 (530, 338)
topadd =  96 96
top_to_pad (530, 96) (530, 96)
-----file name: 5525 176
Datasets/HMSS/imgs/5525
>>> Original width & height: 502 330 (502, 330)
topadd =  86 86
top_to_pad (502, 86) (502, 86)
-----file name: 5526 177
Datasets/HMSS/imgs/5526
>>> Original width & height: 429 307 (429, 307)
topadd =  61 61
top_to_pad (429, 61) (429, 61)
-----file name: 5527 178
Datasets/HMSS/imgs/5527
>>> Original width & height: 420 310 (420, 310)
topadd =  55 55
top_to_pad (420, 55) (420, 55)
-----file name: 5528 179
Datasets/HMSS/imgs/5528
>>> Original width & height: 593 236 (593, 236)
topadd =  178 179
top_to_pad (593, 178) (593, 179)
-----file name: 5529 180
Datasets/HMSS/imgs/5529
>>> Original width & height: 421 287 (421, 287)
topadd =  67 67
top_to_pad (421, 67) (421, 67)
-----file name: 5530 181
Datasets/HMSS/imgs/5530
>>> Original width & height: 420 281 (420, 281)
topadd =  69 70
top_to_pad (420, 69) (420, 70)
-----file name: 5531 182
Datasets/HMS

>>> Original width & height: 554 394 (554, 394)
topadd =  80 80
top_to_pad (554, 80) (554, 80)
-----file name: 5627 235
Datasets/HMSS/imgs/5627
>>> Original width & height: 575 383 (575, 383)
topadd =  96 96
top_to_pad (575, 96) (575, 96)
-----file name: 5628 236
Datasets/HMSS/imgs/5628
>>> Original width & height: 550 378 (550, 378)
topadd =  86 86
top_to_pad (550, 86) (550, 86)
-----file name: 5629 237
Datasets/HMSS/imgs/5629
>>> Original width & height: 575 384 (575, 384)
topadd =  95 96
top_to_pad (575, 95) (575, 96)
-----file name: 5630 238
Datasets/HMSS/imgs/5630
>>> Original width & height: 383 443 (383, 443)
>>> Vertial: left/right pad with 60 cols
-----file name: 5631 239
Datasets/HMSS/imgs/5631
>>> Original width & height: 418 444 (418, 444)
>>> Vertial: left/right pad with 26 cols
-----file name: 5632 240
Datasets/HMSS/imgs/5632
>>> Original width & height: 416 443 (416, 443)
>>> Vertial: left/right pad with 27 cols
-----file name: 5633 241
Datasets/HMSS/imgs/5633
>>> Origin

...

top_to_pad (441, 2) (441, 3)
-----file name: Case-36-U-47-1 45
Datasets/Thammasat/imgs/Case-36-U-47-1
>>> Original width & height: 493 435 (493, 435)
topadd =  29 29
top_to_pad (493, 29) (493, 29)
-----file name: Case-37-U-48-1 46
Datasets/Thammasat/imgs/Case-37-U-48-1
>>> Original width & height: 553 437 (553, 437)
topadd =  58 58
top_to_pad (553, 58) (553, 58)
-----file name: Case-38-U-49-0 47
Datasets/Thammasat/imgs/Case-38-U-49-0
>>> Original width & height: 587 401 (587, 401)
topadd =  93 93
top_to_pad (587, 93) (587, 93)
-----file name: Case-39-U-50-1 48
Datasets/Thammasat/imgs/Case-39-U-50-1
>>> Original width & height: 553 434 (553, 434)
topadd =  59 60
top_to_pad (553, 59) (553, 60)
-----file name: Case-40-U-51-0 49
Datasets/Thammasat/imgs/Case-40-U-51-0
>>> Original width & height: 493 436 (493, 436)
topadd =  28 29
top_to_pad (493, 28) (493, 29)
-----file name: Case-41-U-52-1 50
Datasets/Thammasat/imgs/Case-41-U-52-1
>>> Original width & height: 553 435 (553, 435)
topadd =  

top_to_pad (798, 47) (798, 48)
-----file name: Case-84-U-95-0 94
Datasets/Thammasat/imgs/Case-84-U-95-0
>>> Original width & height: 764 617 (764, 617)
topadd =  73 74
top_to_pad (764, 73) (764, 74)
-----file name: Case-85-U-96-0 95
Datasets/Thammasat/imgs/Case-85-U-96-0
>>> Original width & height: 799 652 (799, 652)
topadd =  73 74
top_to_pad (799, 73) (799, 74)
-----file name: Case-86-U-97-0 96
Datasets/Thammasat/imgs/Case-86-U-97-0
>>> Original width & height: 685 666 (685, 666)
topadd =  9 10
top_to_pad (685, 9) (685, 10)
-----file name: Case-87-U2 97
Datasets/Thammasat/imgs/Case-87-U2
>>> Original width & height: 1186 799 (1186, 799)
topadd =  193 194
top_to_pad (1186, 193) (1186, 194)
-----file name: Case-88-U-1 98
Datasets/Thammasat/imgs/Case-88-U-1
>>> Original width & height: 1077 874 (1077, 874)
topadd =  101 102
top_to_pad (1077, 101) (1077, 102)
-----file name: Case-89-U-1 99
Datasets/Thammasat/imgs/Case-89-U-1
>>> Original width & height: 1280 874 (1280, 874)
topadd =  20

-----file name: Case-134-U2 144
Datasets/Thammasat/imgs/Case-134-U2
>>> Original width & height: 637 867 (637, 867)
>>> Vertial: left/right pad with 230 cols
-----file name: Case-135-U2 145
Datasets/Thammasat/imgs/Case-135-U2
>>> Original width & height: 640 869 (640, 869)
>>> Vertial: left/right pad with 229 cols
-----file name: Case-136-U2 146
Datasets/Thammasat/imgs/Case-136-U2
>>> Original width & height: 636 874 (636, 874)
>>> Vertial: left/right pad with 238 cols
-----file name: Case-137-U2 147
Datasets/Thammasat/imgs/Case-137-U2
>>> Original width & height: 635 866 (635, 866)
>>> Vertial: left/right pad with 231 cols
-----file name: Case-138-U2 148
Datasets/Thammasat/imgs/Case-138-U2
>>> Original width & height: 1281 762 (1281, 762)
topadd =  259 260
top_to_pad (1281, 259) (1281, 260)
-----file name: Case-139-U2 149
Datasets/Thammasat/imgs/Case-139-U2
>>> Original width & height: 638 868 (638, 868)
>>> Vertial: left/right pad with 230 cols
-----file name: Case-140-U2 150
Dataset

-----file name: Case-189-U4 195
Datasets/Thammasat/imgs/Case-189-U4
>>> Original width & height: 785 536 (785, 536)
topadd =  124 125
top_to_pad (785, 124) (785, 125)
-----file name: Case-190-U4 196
Datasets/Thammasat/imgs/Case-190-U4
>>> Original width & height: 691 666 (691, 666)
topadd =  12 13
top_to_pad (691, 12) (691, 13)
-----file name: Case-191-U4 197
Datasets/Thammasat/imgs/Case-191-U4
>>> Original width & height: 864 702 (864, 702)
topadd =  81 81
top_to_pad (864, 81) (864, 81)
-----file name: Case-192-U4 198
Datasets/Thammasat/imgs/Case-192-U4
>>> Original width & height: 864 702 (864, 702)
topadd =  81 81
top_to_pad (864, 81) (864, 81)
-----file name: Case-193-U4 199
Datasets/Thammasat/imgs/Case-193-U4
>>> Original width & height: 988 702 (988, 702)
topadd =  143 143
top_to_pad (988, 143) (988, 143)
-----file name: Case-194-U4 200
Datasets/Thammasat/imgs/Case-194-U4
>>> Original width & height: 737 580 (737, 580)
topadd =  78 79
top_to_pad (737, 78) (737, 79)
-----file name

-----file name: Case-251-U1 245
Datasets/Thammasat/imgs/Case-251-U1
>>> Original width & height: 988 702 (988, 702)
topadd =  143 143
top_to_pad (988, 143) (988, 143)
-----file name: Case-252-U1 246
Datasets/Thammasat/imgs/Case-252-U1
>>> Original width & height: 988 702 (988, 702)
topadd =  143 143
top_to_pad (988, 143) (988, 143)
-----file name: Case-254-U1 247
Datasets/Thammasat/imgs/Case-254-U1
>>> Original width & height: 988 702 (988, 702)
topadd =  143 143
top_to_pad (988, 143) (988, 143)
-----file name: Case-255-U1 248
Datasets/Thammasat/imgs/Case-255-U1
>>> Original width & height: 988 702 (988, 702)
topadd =  143 143
top_to_pad (988, 143) (988, 143)
-----file name: Case-256-U1 249
Datasets/Thammasat/imgs/Case-256-U1
>>> Original width & height: 988 702 (988, 702)
topadd =  143 143
top_to_pad (988, 143) (988, 143)
-----file name: Case-259-U1 250
Datasets/Thammasat/imgs/Case-259-U1
>>> Original width & height: 988 702 (988, 702)
topadd =  143 143
top_to_pad (988, 143) (988, 143

In [59]:
print(type(names_list[1]))

<class 'list'>


# Multitask dataset

We build a multitask dataset at 'Benchmark\Datasets\Multitask' directory and the folder contains two subfolders, B-mode BUS images (imgs), and segmentation ground truth (masks), and a csv file that contains classification ground truth. 

In [60]:
save_csv_benchmark_multi = 'Datasets\Multitask'
if not os.path.exists(save_csv_benchmark_multi):
    os.makedirs(save_csv_benchmark_multi)

names_list=[imgs_name_busis,imgs_name_busi]
paths_list=[imgs_path_busis,imgs_path_busi]
tumor_type_list=[tumor_type_busis,tumor_type_busi]

df_busis = pd.DataFrame(list(zip(imgs_name_busis, tumor_type_busis)),
                        columns=['img name',  'tumor type'])
df_busi = pd.DataFrame(list(zip(imgs_name_busi, tumor_type_busi)),
                       columns=['img name',  'tumor type'])

dfsb = df_busis.append(df_busi,ignore_index=True)          
#dfsb = dfsb.append(df_busi,ignore_index=True) 
     
dfsb['tumor type'] = dfsb["tumor type"].apply(lambda x: 0 if x == 0 or x == "0" or x ==  "B" or x =="b" or x == "benign" or x =="Benign" else 1)       
dfsb['img name'] = dfsb['img name']
dfsb.to_csv(os.path.join(save_csv_benchmark_multi, 'Multitask.csv'), index=False)
print(dfsb[290:300])

     img name  tumor type
290  case0291           0
291  case0292           0
292  case0293           0
293  case0294           0
294  case0295           0
295  case0296           0
296  case0297           0
297  case0298           0
298  case0299           0
299  case0300           0


C:\Users\shar0416\AppData\Local\Temp\ipykernel_20780\406750815.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfsb = df_busis.append(df_busi,ignore_index=True)


In [36]:
def transfer_img( imagesList1, imagesList2, path_img,path_mask,saved_img,saved_mask,dataset_name):
    if (dataset_name == 'BUSI'):
        for imageName in imagesList1:
            img_obj = Image.open(os.path.join(path_img, imageName ))
            img_obj.save(os.path.join(saved_img, imageName ))
        for imageName in imagesList2:
            mask_obj = Image.open(os.path.join(path_mask, imageName ))
            mask_obj.save(os.path.join(saved_mask, imageName[:-9]+'.png' ))
    else:
        for imageName in imagesList1:
            img_obj = Image.open(os.path.join(path_img, imageName ))
            img_obj.save(os.path.join(saved_img, imageName ))
            
        for imageName in imagesList2:
            mask_obj = Image.open(os.path.join(path_mask, imageName ))
            mask_obj.save(os.path.join(saved_mask, imageName[:-7]+'.png' ))



In [67]:
Path_busis_imgs = 'Datasets/BUSIS/imgs/'
Path_busis_GT = 'Datasets/BUSIS/GT/'

Path_busi_imgs = 'Datasets/BUSI/BUSI_combined/imgs/'
Path_busi_masks = 'Datasets/BUSI/BUSI_combined/masks/'

combined_img_savepath_img = 'Datasets/Multitask/imgs/'
combined_img_savepath_mask = 'Datasets/Multitask/mask/'


if not os.path.exists(combined_img_savepath_img):
    os.makedirs(combined_img_savepath_img)

if not os.path.exists(combined_img_savepath_mask):
    os.makedirs(combined_img_savepath_mask)

imagesList1 = listdir(Path_busi_imgs)
imagesList2 = listdir(Path_busi_masks)

imagesList_busis = listdir(Path_busis_imgs)
maskList_busis = listdir(Path_busis_GT)




transfer_img(imagesList_busis, maskList_busis, Path_busis_imgs,Path_busis_GT, combined_img_savepath_img, combined_img_savepath_mask,'BUSIS')
transfer_img(imagesList1, imagesList2, Path_busi_imgs,Path_busi_masks,combined_img_savepath_img, combined_img_savepath_mask,'BUSI')



647 647


### Padding Multitask dataset

In [101]:
def preprocess_multi(imgs_name, imgs_path):
    for b in range(len(imgs_name)):
        print(len(imgs_name))
        filename = imgs_name[b]
        print("-----file name:", filename, b)
        print(imgs_path)
        img_obj = Image.open(os.path.join(imgs_path+filename))
        img_np = np.array(img_obj)      
        #Remove white boundary of the image
        #img_np1 = img_np[1:-1, 1:-1]
        img_np2 = Image.fromarray(img_np)

        width, height = img_np2.size
        print(">>> Original width & height:", width, height, img_obj.size)

        dif = abs(width-height)
        if width-1 > height:
            topadd = int(dif/2)
            bottomadd = dif - topadd
            print("topadd = ", topadd, bottomadd)

            top_to_pad = Image.fromarray(np.zeros((topadd, width, 3), dtype= 'uint8'))
            bottom_to_pad = Image.fromarray(np.zeros((bottomadd, width,  3), dtype = 'uint8'))
            print("top_to_pad",top_to_pad.size, bottom_to_pad.size)

            img_obj_padded = Pad_top_bottom(img_np2, top_to_pad, bottom_to_pad, width)

        elif width < height-1:
            print(">>> Vertial: left/right pad with {} cols".format(dif))
            if dif == abs(1):
                leftadd = int(dif)
                rightadd = dif - leftadd
                
                left_to_pad = Image.fromarray(np.zeros((height, leftadd, 3), dtype = 'uint8'))
                #right_to_pad = Image.fromarray(np.zeros((height, rightadd, 3), dtype = 'uint8'))

                img_obj_padded = Pad_left_right(img_np2, left_to_pad, 0, height)

            else:
                leftadd = int(dif/2)
                rightadd = dif - leftadd
                
                left_to_pad = Image.fromarray(np.zeros((height, leftadd, 3), dtype = 'uint8'))
                right_to_pad = Image.fromarray(np.zeros((height, rightadd, 3), dtype = 'uint8'))

                img_obj_padded = Pad_left_right(img_np2, left_to_pad, right_to_pad, height)

        
        elif width == height-1: # just pad the single col to the right
            print(">>> Vertial: right pad with {} cols".format(dif))
            right_to_pad = Image.fromarray(np.zeros((height, 1, 3), dtype = 'uint8'))
            img_obj_padded = Pad_right(img_np2, right_to_pad, height)
            print(">>> After padding: ", img_obj_padded.size)

        else:
            print(">>> Originally square shape")
            img_obj_padded = img_np2

        filenames = os.path.splitext(filename)[0]
        img_obj_padded.save(os.path.join(imgs_path,filenames + '.png'))   

In [102]:
names_imgs = listdir(combined_img_savepath_img) 
names_mask = listdir(combined_img_savepath_mask) 


preprocess_multi(names_imgs, combined_img_savepath_img)
preprocess_multi(names_mask, combined_img_savepath_mask)

1209
-----file name: benign (1).png 0
Datasets/Multitask/imgs/
>>> Original width & height: 562 471 (562, 471)
topadd =  45 46
top_to_pad (562, 45) (562, 46)
1209
-----file name: benign (10).png 1
Datasets/Multitask/imgs/
>>> Original width & height: 683 585 (683, 585)
topadd =  49 49
top_to_pad (683, 49) (683, 49)
1209
-----file name: benign (100).png 2
Datasets/Multitask/imgs/
>>> Original width & height: 323 473 (323, 473)
>>> Vertial: left/right pad with 150 cols
1209
-----file name: benign (101).png 3
Datasets/Multitask/imgs/
>>> Original width & height: 563 473 (563, 473)
topadd =  45 45
top_to_pad (563, 45) (563, 45)
1209
-----file name: benign (102).png 4
Datasets/Multitask/imgs/
>>> Original width & height: 634 610 (634, 610)
topadd =  12 12
top_to_pad (634, 12) (634, 12)
1209
-----file name: benign (103).png 5
Datasets/Multitask/imgs/
>>> Original width & height: 766 585 (766, 585)
topadd =  90 91
top_to_pad (766, 90) (766, 91)
1209
-----file name: benign (104).png 6
Datasets

1209
-----file name: benign (145).png 51
Datasets/Multitask/imgs/
>>> Original width & height: 779 574 (779, 574)
topadd =  102 103
top_to_pad (779, 102) (779, 103)
1209
-----file name: benign (146).png 52
Datasets/Multitask/imgs/
>>> Original width & height: 449 594 (449, 594)
>>> Vertial: left/right pad with 145 cols
1209
-----file name: benign (147).png 53
Datasets/Multitask/imgs/
>>> Original width & height: 564 463 (564, 463)
topadd =  50 51
top_to_pad (564, 50) (564, 51)
1209
-----file name: benign (148).png 54
Datasets/Multitask/imgs/
>>> Original width & height: 567 463 (567, 463)
topadd =  52 52
top_to_pad (567, 52) (567, 52)
1209
-----file name: benign (149).png 55
Datasets/Multitask/imgs/
>>> Original width & height: 557 468 (557, 468)
topadd =  44 45
top_to_pad (557, 44) (557, 45)
1209
-----file name: benign (15).png 56
Datasets/Multitask/imgs/
>>> Original width & height: 701 526 (701, 526)
topadd =  87 88
top_to_pad (701, 87) (701, 88)
1209
-----file name: benign (150).pn

top_to_pad (557, 41) (557, 42)
1209
-----file name: benign (193).png 104
Datasets/Multitask/imgs/
>>> Original width & height: 754 580 (754, 580)
topadd =  87 87
top_to_pad (754, 87) (754, 87)
1209
-----file name: benign (194).png 105
Datasets/Multitask/imgs/
>>> Original width & height: 711 492 (711, 492)
topadd =  109 110
top_to_pad (711, 109) (711, 110)
1209
-----file name: benign (195).png 106
Datasets/Multitask/imgs/
>>> Original width & height: 729 611 (729, 611)
topadd =  59 59
top_to_pad (729, 59) (729, 59)
1209
-----file name: benign (196).png 107
Datasets/Multitask/imgs/
>>> Original width & height: 766 580 (766, 580)
topadd =  93 93
top_to_pad (766, 93) (766, 93)
1209
-----file name: benign (197).png 108
Datasets/Multitask/imgs/
>>> Original width & height: 547 465 (547, 465)
topadd =  41 41
top_to_pad (547, 41) (547, 41)
1209
-----file name: benign (198).png 109
Datasets/Multitask/imgs/
>>> Original width & height: 759 571 (759, 571)
topadd =  94 94
top_to_pad (759, 94) (75

1209
-----file name: benign (239).png 155
Datasets/Multitask/imgs/
>>> Original width & height: 560 473 (560, 473)
topadd =  43 44
top_to_pad (560, 43) (560, 44)
1209
-----file name: benign (24).png 156
Datasets/Multitask/imgs/
>>> Original width & height: 556 467 (556, 467)
topadd =  44 45
top_to_pad (556, 44) (556, 45)
1209
-----file name: benign (240).png 157
Datasets/Multitask/imgs/
>>> Original width & height: 556 473 (556, 473)
topadd =  41 42
top_to_pad (556, 41) (556, 42)
1209
-----file name: benign (241).png 158
Datasets/Multitask/imgs/
>>> Original width & height: 562 471 (562, 471)
topadd =  45 46
top_to_pad (562, 45) (562, 46)
1209
-----file name: benign (242).png 159
Datasets/Multitask/imgs/
>>> Original width & height: 556 474 (556, 474)
topadd =  41 41
top_to_pad (556, 41) (556, 41)
1209
-----file name: benign (243).png 160
Datasets/Multitask/imgs/
>>> Original width & height: 561 473 (561, 473)
topadd =  44 44
top_to_pad (561, 44) (561, 44)
1209
-----file name: benign (

1209
-----file name: benign (286).png 207
Datasets/Multitask/imgs/
>>> Original width & height: 620 490 (620, 490)
topadd =  65 65
top_to_pad (620, 65) (620, 65)
1209
-----file name: benign (287).png 208
Datasets/Multitask/imgs/
>>> Original width & height: 466 390 (466, 390)
topadd =  38 38
top_to_pad (466, 38) (466, 38)
1209
-----file name: benign (288).png 209
Datasets/Multitask/imgs/
>>> Original width & height: 463 390 (463, 390)
topadd =  36 37
top_to_pad (463, 36) (463, 37)
1209
-----file name: benign (289).png 210
Datasets/Multitask/imgs/
>>> Original width & height: 468 393 (468, 393)
topadd =  37 38
top_to_pad (468, 37) (468, 38)
1209
-----file name: benign (29).png 211
Datasets/Multitask/imgs/
>>> Original width & height: 561 462 (561, 462)
topadd =  49 50
top_to_pad (561, 49) (561, 50)
1209
-----file name: benign (290).png 212
Datasets/Multitask/imgs/
>>> Original width & height: 465 389 (465, 389)
topadd =  38 38
top_to_pad (465, 38) (465, 38)
1209
-----file name: benign (

...

>>> Original width & height: 708 428 (708, 428)
topadd =  140 140
top_to_pad (708, 140) (708, 140)
1209
-----file name: case0535.png 971
Datasets/Multitask/mask/
>>> Original width & height: 649 515 (649, 515)
topadd =  67 67
top_to_pad (649, 67) (649, 67)
1209
-----file name: case0536.png 972
Datasets/Multitask/mask/
>>> Original width & height: 655 550 (655, 550)
topadd =  52 53
top_to_pad (655, 52) (655, 53)
1209
-----file name: case0537.png 973
Datasets/Multitask/mask/
>>> Original width & height: 708 554 (708, 554)
topadd =  77 77
top_to_pad (708, 77) (708, 77)
1209
-----file name: case0538.png 974
Datasets/Multitask/mask/
>>> Original width & height: 557 368 (557, 368)
topadd =  94 95
top_to_pad (557, 94) (557, 95)
1209
-----file name: case0539.png 975
Datasets/Multitask/mask/
>>> Original width & height: 553 353 (553, 353)
topadd =  100 100
top_to_pad (553, 100) (553, 100)
1209
-----file name: case0540.png 976
Datasets/Multitask/mask/
>>> Original width & height: 550 348 (550, 3

>>> Original width & height: 569 458 (569, 458)
topadd =  55 56
top_to_pad (569, 55) (569, 56)
1209
-----file name: malignant (13).png 1033
Datasets/Multitask/mask/
>>> Original width & height: 468 393 (468, 393)
topadd =  37 38
top_to_pad (468, 37) (468, 38)
1209
-----file name: malignant (130).png 1034
Datasets/Multitask/mask/
>>> Original width & height: 555 463 (555, 463)
topadd =  46 46
top_to_pad (555, 46) (555, 46)
1209
-----file name: malignant (131).png 1035
Datasets/Multitask/mask/
>>> Original width & height: 604 461 (604, 461)
topadd =  71 72
top_to_pad (604, 71) (604, 72)
1209
-----file name: malignant (132).png 1036
Datasets/Multitask/mask/
>>> Original width & height: 451 581 (451, 581)
>>> Vertial: left/right pad with 130 cols
1209
-----file name: malignant (133).png 1037
Datasets/Multitask/mask/
>>> Original width & height: 479 485 (479, 485)
>>> Vertial: left/right pad with 6 cols
1209
-----file name: malignant (134).png 1038
Datasets/Multitask/mask/
>>> Original widt

-----file name: malignant (178).png 1086
Datasets/Multitask/mask/
>>> Original width & height: 562 467 (562, 467)
topadd =  47 48
top_to_pad (562, 47) (562, 48)
1209
-----file name: malignant (179).png 1087
Datasets/Multitask/mask/
>>> Original width & height: 432 476 (432, 476)
>>> Vertial: left/right pad with 44 cols
1209
-----file name: malignant (18).png 1088
Datasets/Multitask/mask/
>>> Original width & height: 579 437 (579, 437)
topadd =  71 71
top_to_pad (579, 71) (579, 71)
1209
-----file name: malignant (180).png 1089
Datasets/Multitask/mask/
>>> Original width & height: 562 461 (562, 461)
topadd =  50 51
top_to_pad (562, 50) (562, 51)
1209
-----file name: malignant (181).png 1090
Datasets/Multitask/mask/
>>> Original width & height: 497 488 (497, 488)
topadd =  4 5
top_to_pad (497, 4) (497, 5)
1209
-----file name: malignant (182).png 1091
Datasets/Multitask/mask/
>>> Original width & height: 689 583 (689, 583)
topadd =  53 53
top_to_pad (689, 53) (689, 53)
1209
-----file name:

>>> Original width & height: 592 387 (592, 387)
topadd =  102 103
top_to_pad (592, 102) (592, 103)
1209
-----file name: malignant (36).png 1139
Datasets/Multitask/mask/
>>> Original width & height: 623 494 (623, 494)
topadd =  64 65
top_to_pad (623, 64) (623, 65)
1209
-----file name: malignant (37).png 1140
Datasets/Multitask/mask/
>>> Original width & height: 626 490 (626, 490)
topadd =  68 68
top_to_pad (626, 68) (626, 68)
1209
-----file name: malignant (38).png 1141
Datasets/Multitask/mask/
>>> Original width & height: 1048 578 (1048, 578)
topadd =  235 235
top_to_pad (1048, 235) (1048, 235)
1209
-----file name: malignant (39).png 1142
Datasets/Multitask/mask/
>>> Original width & height: 564 469 (564, 469)
topadd =  47 48
top_to_pad (564, 47) (564, 48)
1209
-----file name: malignant (4).png 1143
Datasets/Multitask/mask/
>>> Original width & height: 432 473 (432, 473)
>>> Vertial: left/right pad with 41 cols
1209
-----file name: malignant (40).png 1144
Datasets/Multitask/mask/
>>> O

top_to_pad (733, 124) (733, 124)
1209
-----file name: malignant (86).png 1194
Datasets/Multitask/mask/
>>> Original width & height: 735 483 (735, 483)
topadd =  126 126
top_to_pad (735, 126) (735, 126)
1209
-----file name: malignant (87).png 1195
Datasets/Multitask/mask/
>>> Original width & height: 561 499 (561, 499)
topadd =  31 31
top_to_pad (561, 31) (561, 31)
1209
-----file name: malignant (88).png 1196
Datasets/Multitask/mask/
>>> Original width & height: 775 591 (775, 591)
topadd =  92 92
top_to_pad (775, 92) (775, 92)
1209
-----file name: malignant (89).png 1197
Datasets/Multitask/mask/
>>> Original width & height: 786 585 (786, 585)
topadd =  100 101
top_to_pad (786, 100) (786, 101)
1209
-----file name: malignant (9).png 1198
Datasets/Multitask/mask/
>>> Original width & height: 430 480 (430, 480)
>>> Vertial: left/right pad with 50 cols
1209
-----file name: malignant (90).png 1199
Datasets/Multitask/mask/
>>> Original width & height: 783 587 (783, 587)
topadd =  98 98
top_to_